### Investigating use of KF to recover parameters

In [3]:
import os
from ModularCirc.Models.KorakianitisMixedModel import KorakianitisMixedModel, KorakianitisMixedModel_parameters, TEMPLATE_TIME_SETUP_DICT
from ModularCirc import BatchRunner
import numpy as np
import pandas as pd
from cvdnet_pipeline.utils import helper_functions, plot_utils


1. Sample parameters, 100 beats, pat.r gradually increasing/step function

In [ ]:
# Setup
param_path = "input_parameters_jsons/parameters_patr_drift.JSON"
output_path = "output_synthetic"
n_samples = 100
repeat_simulations = False

# Sample parameters or read in from file
br = BatchRunner('Sobol', 0)
br.setup_sampler(param_path)
br.sample(n_samples)

######## 

# pat.r with step function drift
pat_r = np.zeros((n_samples,))
for i in range(5):       
     pat_r[i*20:(i+1)*20] = 0.3 + i*0.1
 
#pat_r = pat_r + np.random.normal(0, 0.05, n_samples)
pat_r = pd.DataFrame(pat_r, columns=['pat_r'])

br._samples['pat.r'] = pat_r['pat_r'].

########



/Users/pmzff/Documents/GitHub/cvd-net-pipeline/cvd-net-pipeline/venv/lib/python3.13/site-packages/ModularCirc/_BatchRunner.py:59: UserWarning: The balance properties of Sobol' points require n to be a power of 2.
  samples  = self._sampler.random(nsamples)


Running simulation as output_synthetic/output_100_2_params.


100%|██████████| 100/100 [00:34<00:00,  2.90it/s]


2. Simulate beats

In [ ]:
# Get relevant columns  
relevant_columns = [col for col in br._samples.columns if col in br._parameters_2_sample.keys()]
_pure_samples = br._samples[relevant_columns].copy()


map_ = {
        'delay': ['la.delay', 'ra.delay'],
        'td0': ['lv.td0', 'rv.td0'],
        'tr': ['lv.tr', 'rv.tr'],
        'tpww': ['la.tpww', 'ra.tpww'],
    }
    
br.map_sample_timings(
    ref_time=1.,
    map=map_
)

br._samples[['lv.td', 'rv.td']] = br._samples[['lv.tr', 'rv.tr']].values + br._samples[['lv.td0', 'rv.td0']].values
br._samples.drop(['lv.td0', 'rv.td0'], axis=1, inplace=True)

# get relevant columns and count them
n_params = len(relevant_columns)
br.map_vessel_volume()

br.setup_model(model=KorakianitisMixedModel, po=KorakianitisMixedModel_parameters,
                time_setup=TEMPLATE_TIME_SETUP_DICT)

file_suffix  = f'_{n_samples}_{n_params}_params'

# Save samples as CSV files 
_pure_samples.to_csv(os.path.join(output_path, f'pure_input{file_suffix}.csv'), index=False)
br.samples.to_csv(os.path.join(output_path, f'input{file_suffix}.csv'), index=False)

output_parameters = os.path.join(output_path, f'output{file_suffix}')
output_parameters_simulations = os.path.join(output_parameters,'simulations')


# Check if the directory exists and contains n_samples files
if os.path.exists(output_parameters_simulations) and len(os.listdir(output_parameters_simulations)) >= n_samples and repeat_simulations==False:
    print(f"Skipping simulation as {output_parameters} already contains {n_samples} or more files.")
    # read a list of dataframes from here
    simulations = helper_functions.load_simulation(output_parameters_simulations)

    if len(simulations) != n_samples:
        raise ValueError(f"Expected {n_samples} simulations, but found {len(simulations)}. Will run simulations again.")
        repeat_simulations = True
else:
    print(f"Running simulation as {output_parameters}.")
    repeat_simulations = True


if repeat_simulations:
    os.makedirs(output_parameters_simulations, exist_ok=True)
    simulations = br.run_batch(
        n_jobs=7,
        output_path=output_parameters_simulations
    )